In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

In [3]:
torch.__version__


'2.3.1'

In [7]:
#hypyer parameters 
batch_size = 64
block_size = 256
max_iters = 5000

eval_interval = 500
learning_rate = 3e-4
device ='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

n_embs = 384
n_head = 6 
n_layer = 6
dropout = 0.2

In [8]:
torch.manual_seed(1337)

In [9]:
with open('../input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [22]:
# crate vocabulary list (sorted)
char = sorted(list(set(text)))
vocab_size = len(char)
vocab_size

65

In [28]:
stoi = {char: i for i, char in enumerate(char)}
itos = {i: char for i, char in enumerate(char)}


In [38]:
b = encoding('fuck you!')
b

[44, 59, 41, 49, 1, 63, 53, 59, 2]

In [65]:
encoding = lambda word: [stoi[i] for i in word]
# use '' to concatenate str 
decoding = lambda number: ''.join(itos[i] for i in number)

In [97]:
#training ans tesing split 
data = torch.tensor(encoding(text), dtype=torch.int64)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [201]:
# Data loading 
# Batch in training and validation
def get_batch(set_name):
    data = (train_data if set_name == 'train' else  val_data)
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y

In [202]:
data_x, data_y = get_batch('train')


In [203]:
data_x.shape, data_y.shape


(torch.Size([64, 256]), torch.Size([64, 256]))